In [ ]:
!python -m spacy download pt_core_news_sm
!pip install googletrans==4.0.0-rc1
!pip install enelvo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.6 MB/s eta 0:

In [ ]:
from google.colab import drive
from googletrans import Translator
from enelvo.normaliser import Normaliser
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Dense, Concatenate, Input, LSTM, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import re
import numpy as np
import pandas as pd
import spacy
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Pré-processamento

## 0) Base de Dados

In [ ]:
norm = Normaliser(tokenizer='readable')
chats_clients = pd.read_csv('/content/drive/Shareddrives/grupo3moshi/chats_clients.csv')
chats_clients.head()

,No,Intencao,Pergunta,Resposta
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [ ]:
chats_clients.fillna('erro ao processar a pergunta', inplace=True)

In [ ]:
nlp_pt = spacy.load("pt_core_news_sm")

In [ ]:
def correct_grammar(text):
  return norm.normalise(text)

## 1) Word2Vec

O processo de Word2Vec é uma técnica de aprendizado de máquina com o objetivo de vetorizar palavras almejando preservar o contexto, convertendo o resultado para a forma numérica, para que os dados possam ser utilizados em modelos de aprendizado de máquina.

O Word2Vec pode possuir 2 arquiteturas, sendo elas:

- Skip-Gram: rede tenta prever as palavras de contexto ao redor de uma palavra central
- CBOW: rede tenta prever a palavra central com base no contexto ao redor da palavra central.

Para fins de comparação, ambas as técnicas serão abordadas no código a seguir:

### 1.1) Skip-Gram

In [ ]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenizando as colunas
chats_clients['Pergunta_tokens'] = chats_clients['Pergunta'].apply(lambda x: word_tokenize(x.lower()))
chats_clients['Resposta_tokens'] = chats_clients['Resposta'].apply(lambda x: word_tokenize(x.lower()))

# Unindo as listas de tokens de Pergunta e Resposta para o modelo processar a lista
sentences_skip = chats_clients['Pergunta_tokens'].tolist() + chats_clients['Resposta_tokens'].tolist()

# Utilizando o modelo Word2Vec
model = Word2Vec(sentences_skip, vector_size=100, window=5, min_count=1, sg=1)  #sg=1 para Skip-Gram

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 1.2) Resultados Obtidos

In [ ]:
# Acessando o vetor de uma palavra
vector_dia_skip = model.wv['dia']
print(vector_dia_skip)

[-0.14521278  0.0699736  -0.19564913 -0.19521366  0.08546572 -0.19507958
  0.30101863  0.3439376   0.09365555 -0.35656157  0.0657161   0.01176258
  0.0522949  -0.17894548  0.1002083   0.0201362  -0.00313176 -0.03761178
 -0.08768682 -0.2266451   0.19865747 -0.04156136  0.21142502 -0.03225799
 -0.15834308  0.00198257 -0.2745638  -0.06593608  0.12919365 -0.05153619
  0.06584648  0.1723829  -0.22677952  0.08527343  0.08272042  0.17869793
 -0.03245538  0.06444605 -0.15435399 -0.10113306  0.3924239  -0.06792287
 -0.1317383   0.03544988  0.3956842   0.20504841 -0.37416857 -0.12702037
  0.09101572  0.14444172  0.13409898 -0.24576643  0.0604369  -0.14947854
 -0.03500679  0.1453391  -0.0409093   0.13835624 -0.11010893  0.07222822
  0.13237613 -0.01449497  0.02134003 -0.01710702  0.05616313 -0.05496536
  0.19621642  0.2424914  -0.15521955  0.09669434 -0.32146773  0.04742014
  0.22986533 -0.00289251  0.01731809 -0.06804271  0.02029018  0.08611777
 -0.22065377  0.14743342 -0.07412688 -0.04069695 -0

In [ ]:
# Encontrando palavras similares
similar_words_dia_skip = model.wv.most_similar('dia')
print(similar_words_dia_skip)

[('bom', 0.9674476385116577), ('noite', 0.9415793418884277), ('olá', 0.9347184300422668), ('boa', 0.9203506708145142), ('!', 0.9150063395500183), ('fiz', 0.9108341336250305), ('uma', 0.9101818799972534), ('tarde', 0.908665120601654), ('vcs', 0.9060782194137573), ('tenho', 0.9060748815536499)]


In [ ]:
# Encontrando palavras similares
similar_words_bom_skip = model.wv.most_similar('bom')
print(similar_words_bom_skip)

[('dia', 0.9674476981163025), ('noite', 0.949826717376709), ('!', 0.9382209181785583), ('boa', 0.9368942379951477), ('olá', 0.9304441213607788), ('tarde', 0.9268419742584229), ('uma', 0.9229543209075928), ('informar', 0.9088412523269653), ('fiz', 0.9056987762451172), ('oi', 0.8998782634735107)]


In [ ]:
# Encontrando palavras similares
similar_words_dinheiro_skip = model.wv.most_similar('dinheiro')
print(similar_words_dinheiro_skip)

[('japão', 0.9774123430252075), ('tempo', 0.9751770496368408), ('?', 0.9732164740562439), ('pra', 0.972236692905426), ('enviar', 0.9654679894447327), ('levará', 0.9617966413497925), ('como', 0.9538540244102478), ('mandar', 0.949092447757721), ('eu', 0.9469629526138306), ('qual', 0.9459159970283508)]


In [ ]:
# Encontrando palavras similares
similar_words_pix_skip = model.wv.most_similar('pix')
print(similar_words_pix_skip)

[('via', 0.9896284341812134), ('pagamentos', 0.9778498411178589), ('feitos', 0.9749538898468018), ('sendo', 0.9597593545913696), ('estão', 0.9544371366500854), ('pagas', 0.9402541518211365), ('dentro', 0.9211169481277466), ('são', 0.8903807997703552), ('realizadas', 0.879164457321167), ('algumas', 0.866555392742157)]


### 2.1) CBOW

In [ ]:
# Unindo as listas de tokens de Pergunta e Resposta para o modelo processar a lista
sentences_CBOW = chats_clients['Pergunta_tokens'].tolist() + chats_clients['Resposta_tokens'].tolist()

# Utilizando o modelo Word2Vec
model_cbow = Word2Vec(sentences_CBOW, vector_size=100, window=5, min_count=1, sg=0)  #sg=0 para CBOW

### 2.2) Resultados Obtidos

In [ ]:
# Acessando o vetor de uma palavra
vector_dia_cbow = model_cbow.wv['dia']
print(vector_dia_cbow)

[-0.21853107  0.3343532  -0.04802862 -0.02650334  0.1443724  -0.64473957
  0.29255304  0.8815682  -0.3291454  -0.33276838 -0.18376827 -0.4858411
  0.05904582 -0.08773582  0.23664865 -0.14113839  0.14197081 -0.40860268
 -0.21815738 -0.82510304  0.08361284  0.12890868  0.24721539 -0.18441725
 -0.18082042 -0.03896096 -0.2582402  -0.32055745 -0.34191516  0.20226367
  0.4141003   0.0657732   0.12284658 -0.2870549  -0.2578394   0.5911866
 -0.01721214 -0.2367317  -0.14286558 -0.68978924  0.29506525 -0.25213742
 -0.19745693  0.02772277  0.40023884  0.00179859 -0.39361632 -0.11996371
  0.20846182  0.17235987  0.26469305 -0.44942957  0.05395574 -0.08779406
 -0.30510607  0.13944449  0.13553485 -0.01869507 -0.38441497  0.23487973
  0.16007684  0.01112592  0.052938   -0.14018686 -0.33248326  0.25995645
  0.27446258  0.3535761  -0.53066427  0.4360944  -0.37967873  0.10513819
  0.32137454  0.04305624  0.31028974  0.14653252  0.01229515 -0.00958467
 -0.48743242  0.12336485 -0.10764622  0.12234768 -0.5

In [ ]:
# Encontrando palavras similares
similar_words_dia_cbow = model_cbow.wv.most_similar('dia')
print(similar_words_dia_cbow)

[('quanto', 0.9992035031318665), ('fiz', 0.9991766810417175), ('eu', 0.9991341233253479), ('uma', 0.9991209506988525), ('!', 0.9990766644477844), ('um', 0.9990658164024353), ('gostaria', 0.9990556836128235), ('em', 0.9990227222442627), ('saber', 0.9989913105964661), ('pode', 0.9989619255065918)]


In [ ]:
# Encontrando palavras similares
similar_words_bom_cbow = model_cbow.wv.most_similar('bom')
print(similar_words_bom_cbow)

[('uma', 0.9992238879203796), ('um', 0.999219536781311), ('está', 0.9991664886474609), ('eu', 0.9991341829299927), ('meu', 0.9991222023963928), ('enviar', 0.9990976452827454), ('fiz', 0.9990931153297424), ('em', 0.9990674257278442), ('!', 0.9990590214729309), ('quanto', 0.9990459680557251)]


In [ ]:
# Encontrando palavras similares
similar_words_dinheiro_cbow = model_cbow.wv.most_similar('dinheiro')
print(similar_words_dinheiro_cbow)

[('enviar', 0.9995072484016418), ('?', 0.9995015859603882), ('pra', 0.9994408488273621), ('do', 0.9994198679924011), ('quanto', 0.9993830919265747), ('beneficiário', 0.9993560314178467), ('valor', 0.9993547797203064), ('uma', 0.9993402361869812), ('eu', 0.9993310570716858), ('como', 0.9993306994438171)]


In [ ]:
# Encontrando palavras similares
similar_words_pix_cbow = model_cbow.wv.most_similar('pix')
print(similar_words_pix_cbow)

[('via', 0.9991001486778259), ('as', 0.9989967346191406), ('dos', 0.998813271522522), ('estão', 0.9987587332725525), ('pagas', 0.9987448453903198), ('~', 0.9987034797668457), ('tudo', 0.9986793994903564), ('foto', 0.9986681342124939), ('sendo', 0.9986535310745239), ('que', 0.9986518621444702)]


### 3.1) Word Embedding


Dessa forma, é fundamental gerar os embedding das colunas para que futuros modelos de aprendizado de máquina possam compreender e utilizar os dados de cada frase. Para tal, será utilizado uma média dos vetores de cada frase, desenvolvido a seguir, utilizando a abordagem CBOW (padrão):


In [ ]:
import numpy as np

# Função para gerar o embedding médio de uma frase
def get_sentence_embedding(tokens, model):
    word_vectors = [model.wv[token] for token in tokens if token in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

### 3.2) Resultados Obtidos

In [ ]:
# Aplicando a função para gerar os embeddings e salvando nas novas colunas
chats_clients['Pergunta_embedding'] = chats_clients['Pergunta_tokens'].apply(lambda tokens: get_sentence_embedding(tokens, model_cbow))
chats_clients['Resposta_embedding'] = chats_clients['Resposta_tokens'].apply(lambda tokens: get_sentence_embedding(tokens, model_cbow))

In [ ]:
chats_clients['Pergunta_embedding'].head()

,Pergunta_embedding
0,"[-0.16774349, 0.22888495, -0.024799801, -0.001..."
1,"[-0.2580988, 0.33707348, -0.04060621, -0.01579..."
2,"[-0.23581156, 0.3286497, -0.041849144, -0.0016..."
3,"[-0.19775629, 0.2726439, -0.034863938, -0.0075..."
4,"[-0.167257, 0.2393431, -0.022884179, 0.0055263..."


In [ ]:
chats_clients['Resposta_embedding'].head()

,Resposta_embedding
0,"[-0.15577121, 0.22501503, -0.0260906, 0.002611..."
1,"[-0.17000939, 0.32869914, -0.036122166, 0.0324..."
2,"[-0.17597212, 0.28376278, -0.03570332, 0.00588..."
3,"[-0.15591788, 0.2581829, -0.020867042, 0.02739..."
4,"[-0.15591788, 0.2581829, -0.020867042, 0.02739..."


## 2) Base de Dados Final

In [ ]:
chats_clients

,No,Intencao,Pergunta,Resposta,Pergunta_tokens,Resposta_tokens,Pergunta_embedding,Resposta_embedding
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d...","[boa, dia.tudo, bem, ?, eu, gostaria, de, sabe...","[bom, dia, !, sim, ,, o, sr, pode, utilizar, o...","[-0.16774349, 0.22888495, -0.024799801, -0.001...","[-0.15577121, 0.22501503, -0.0260906, 0.002611..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f...","[como, enviar, dinheiro, do, japão, ?]","[para, se, inscrever, no, serviço, de, remessa...","[-0.2580988, 0.33707348, -0.04060621, -0.01579...","[-0.17000939, 0.32869914, -0.036122166, 0.0324..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ...","[quanto, tempo, levará, para, o, beneficiário,...","[via, de, regra, ,, as, remessas, serão, pagas...","[-0.23581156, 0.3286497, -0.041849144, -0.0016...","[-0.17597212, 0.28376278, -0.03570332, 0.00588..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...,"[boa, tarde, acabei, de, fazer, a, transferênc...","[iremos, processar, a, sua, solicitacao, ., mu...","[-0.19775629, 0.2726439, -0.034863938, -0.0075...","[-0.15591788, 0.2581829, -0.020867042, 0.02739..."
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...,"[poderia, fazer, a, remessa, de, 22yenes, para...","[iremos, processar, a, sua, solicitacao, ., mu...","[-0.167257, 0.2393431, -0.022884179, 0.0055263...","[-0.15591788, 0.2581829, -0.020867042, 0.02739..."
...,...,...,...,...,...,...,...,...
500,501,Atualizacao de dados cadastrais,Boa tarde como faço para fazer a mudança de en...,A senhora pode trocar pelo próprio app da Bras...,"[boa, tarde, como, faço, para, fazer, a, mudan...","[a, senhora, pode, trocar, pelo, próprio, app,...","[-0.22479777, 0.30782035, -0.037784025, -0.005...","[-0.17825833, 0.2743183, -0.035151236, -0.0022..."
501,502,Tempo de entrega do cartao,Bom dia\nConsegui fazer depósito sem o cartão ...,Boa noite\nO sr solicitou dia 28. Será entregu...,"[bom, dia, consegui, fazer, depósito, sem, o, ...","[boa, noite, o, sr, solicitou, dia, 28., será,...","[-0.18266651, 0.25676706, -0.032218717, 0.0011...","[-0.14644134, 0.2132509, -0.023235392, 0.00212..."
502,503,Cancelamento,"Boa noite,a beneficiária me avisou que a conta...","Sim, podemos cancelar\nO valor voltará no seu ...","[boa, noite, ,, a, beneficiária, me, avisou, q...","[sim, ,, podemos, cancelar, o, valor, voltará,...","[-0.19458148, 0.28377137, -0.031848915, 0.0015...","[-0.17556186, 0.26671132, -0.032258917, 0.0025..."
503,504,Reembolso,"Boa noite\nEu fiz um depósito errado, era pra ...",É possível solicitar o reembolso diretamente d...,"[boa, noite, eu, fiz, um, depósito, errado, ,,...","[é, possível, solicitar, o, reembolso, diretam...","[-0.21939377, 0.3116105, -0.037289187, -0.0041...","[-0.17090175, 0.25133634, -0.022091111, 0.0128..."


In [ ]:
class IntentClassifier:
    def __init__(self, epochs=500, batch_size=16, test_size=0.4, validation_split=0.4, learning_rate=0.001):
        self.epochs = epochs
        self.batch_size = batch_size
        self.test_size = test_size
        self.validation_split = validation_split
        self.learning_rate = learning_rate
        self.model = None
        self.label_encoder = None

    def prepare_data(self, chats_clients):
        answer_embeddings = np.array(chats_clients['Pergunta_embedding'].tolist())
        response_embeddings = np.array(chats_clients['Resposta_embedding'].tolist())

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(chats_clients['Intencao'])

        combined_embeddings = np.stack((answer_embeddings, response_embeddings), axis=1)
        X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, self.labels, test_size=self.test_size, random_state=42)

        return X_train, X_test, y_train, y_test

    def build_model(self, input_shape):
        self.model = Sequential()
        self.model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
        self.model.add(Dropout(0.5))
        self.model.add(LSTM(64))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(len(np.unique(self.labels)), activation='softmax'))
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model.summary()

    def train(self, X_train, X_test, y_train, y_test):
        # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

        self.model.fit(X_train, y_train, epochs=self.epochs,
                       batch_size=self.batch_size,
                       validation_split=self.validation_split,
                       callbacks=[
                          #  early_stopping,
                           model_checkpoint
                           ])

        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        return y_pred_classes

    def generate_report(self, y_test, y_pred_classes):
        precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred_classes)

        report_df = pd.DataFrame({
            'Class': self.label_encoder.classes_,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'Support': support
        })

        report_df.loc['Average'] = report_df[['Precision', 'Recall', 'F1-Score', 'Support']].mean()
        report_df.loc['Average', 'Class'] = 'Average'

        print(report_df)


In [ ]:
classifier = IntentClassifier()
X_train, X_test, y_train, y_test = classifier.prepare_data(chats_clients)
classifier.build_model(input_shape=(X_train.shape[1], X_train.shape[2]))
classifier.train(X_train, X_test, y_train, y_test)
y_pred_classes = classifier.predict(X_test)
classifier.generate_report(y_test, y_pred_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_40 (Bidirectional)     │ (None, 2, 256)              │         234,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 2, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_81 (LSTM)                       │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_120 (Dense)                    │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_121 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_122 (Dense)                    │ (None, 18)                  │             594 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 323,506 (1.23 MB)

 Trainable params: 323,506 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.1163 - loss: 2.8755 - val_accuracy: 0.2459 - val_loss: 2.8268
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2616 - loss: 2.7825 - val_accuracy: 0.2459 - val_loss: 2.7372
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2418 - loss: 2.7049 - val_accuracy: 0.2459 - val_loss: 2.7049
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2625 - loss: 2.5798 - val_accuracy: 0.2459 - val_loss: 2.6657
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.2470 - loss: 2.5245 - val_accuracy: 0.2459 - val_loss: 2.6524
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2923 - loss: 2.3630 - val_accuracy: 0.2459 - val_loss: 2.6559
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2262 - loss: 2.5436 - val_accuracy: 0.2459 - val_loss: 2.6418
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2074 - loss: 2.4843 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
